In [1]:
library("DESeq2")

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which, which.max, which.min



Attaching package: ‘S4Vectors’


The

In [10]:
cts_file='/project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/01_transcriptome/golong_all_2022_12_all_genes_counts.csv'
cts <- as.matrix(read.csv(cts_file,sep=",", row.names='X'))
head(cts,2)

,K01,K02,K03,K04,N01,N02,N03,N04,C01,C02,⋯,M22,M23,M24,M25,B01,B02,B03,B04,B05,B06
ENSG00000278704.1,11,2,13,11,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
PB_novel_05614,0,1,1,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [8]:
samples_file="/project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/01_transcriptome/golong_all_2022_12_samples.csv"
samples=read.csv(samples_file,sep=",",row.names="name")
head(samples,2)

,X,file,group,nonchimeric_reads,chimeric_reads,internal_id,phenotype,genotype,platform,TSM_ID,sample_pool,DNA_smear,illumina_sample_id,SF3B1_mut,SF3B1_AF
,<int>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>
K01,0,/project/42/pacbio/golong/05d-raw_align_mm2/D001_flnc_aligned.bam,K562_wt,694647,314,K562_wt_1,K562,wt,S2,D001,1-1,2828,MRS938,,
K02,0,/project/42/pacbio/golong/05d-raw_align_mm2/D002_flnc_aligned.bam,K562_wt,525087,540,K562_wt_2,K562,wt,S2,D002,1-1,2692,MRS940,,


In [11]:

dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = samples,
                              design = ~ phenotype+genotype)

In [12]:
keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

In [14]:
dds <- DESeq(dds)
res <- results(dds)


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 325 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [35]:
res_wt_mut <- results(dds, contrast=c("genotype","wt","mut"), name='wt vs mut', alpha=0.1)
summary(res_wt_mut)
res_wt_mut <- res_wt_mut[order(res_wt_mut$pvalue),]
subset(res_wt_mut, padj < 0.1)
out_file="/project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/04_differential_splicing/DESeq/golong_all_2022_12_DESeq_wt_mut.csv"

write.csv(as.data.frame(res_wt_mut), file=out_file)


out of 24572 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 7, 0.028%
LFC < 0 (down)     : 8, 0.033%
outliers [1]       : 34, 0.14%
low counts [2]     : 14292, 58%
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



log2 fold change (MLE): genotype wt vs mut 
Wald test p-value: genotype wt vs mut 
DataFrame with 15 rows and 6 columns
                           baseMean     log2FoldChange             lfcSE
                          <numeric>          <numeric>         <numeric>
ENSG00000132704.16 133.523565488384   2.21553549010624 0.427152255411468
ENSG00000267002.4  3.46476354849288  -1.76759242428026 0.348764001912363
ENSG00000145287.11 73.1225894768266   2.63928542604816 0.578394356646864
ENSG00000167232.14 24.4853779780431  -1.27229947678059 0.282376111198743
PB_novel_20766     40.1562418451005  -3.57070889541602 0.802380960213559
...                             ...                ...               ...
ENSG00000245060.8  2.54261919220278  -1.42221460395326 0.369189091980319
ENSG00000143624.14 2.57382732215794   2.65841498980272 0.694713478927013
ENSG00000100941.9  257.904958147878 -0.636025623140091 0.166540450054175
ENSG00000173926.6  11.0103956788007  -1.30945868746366 0.343501127263008
ENSG

In [31]:
res_MDS_CLL <- results(dds, contrast=c("phenotype","MDS","CLL"), name='MDS vs CLL', alpha=0.01)
res_MDS_CLL <- res_MDS_CLL[order(res_MDS_CLL$pvalue),]

In [28]:
summary(res_MDS_CLL)


out of 24572 with nonzero total read count
adjusted p-value < 0.01
LFC > 0 (up)       : 5247, 21%
LFC < 0 (down)     : 2106, 8.6%
outliers [1]       : 34, 0.14%
low counts [2]     : 8579, 35%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [34]:
out_file="/project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/04_differential_splicing/DESeq/golong_all_2022_12_DESeq_MDS_CLL.csv"

write.csv(as.data.frame(res_MDS_CLL), file=out_file)